# Preambule

#### Goal

The goal of this session is to get familiar with the Bloomberg Python API.<br>
This will be done by building a class containing a function which mimicks the behavior of the BDH Excel function.

#### What the function will do

Our BDH-like function should be able to : <br>
1 - Retrieve historical data <br>
2 - For as many tickers as possible <br>
3 - For as many fields as possible <br>
4 - Include the various options <br>
5 - And allow for the possibility to add overrides <br>

#### References

https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf


# I. Dependencies

These are the libraries we will be using in this notebook. blpapi is the library used for Bloomberg data.


In [3]:
import blpapi
import pandas as pd
import numpy as np
import datetime as dt

# II. Set up the Bloomberg names

We here create variables using the Name class within blpapi. <br>
This will allow to write cleaner and more concise code when refering to strings with the api.<br>
Below are only the names required for our present work. Many more exist and you can refer to the different examples within the SDK for ones of interest to your task.


In [7]:
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

# III. The BLP class

We now start to build our function within a dedicated class.<br>

A brief reminder on the class object in Python:<br>

- Classes must have a function called _\_init_\_() which is automatically executed at class initiation
- Classes can have one or several methods
- Class object need to be instaciated before using its methods

#### A. The init function

This function aims at starting the session and setting up the desired service

#### B. The close session method:

Simply kills the session so no ghost connection remains.

#### C. The BDP method:

3 steps: <br>
1- Create request<br>
2- Send request <br>
3- Extract data<br>


In [159]:
from typing import Literal, Optional


class BLP:
    # -----------------------------------------------------------------------------------------------------
    def __init__(self):
        """
        Improve this
        BLP object initialization
        Synchronus event handling

        """
        # Create Session object
        self.session = blpapi.Session()
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return


        # Open & Get RefData Service or exit if impossible

        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return


        self.session.openService("//BLP/refdata")

        self.refDataSvc = self.session.getService("//BLP/refdata")

        print("Session open")


    # -----------------------------------------------------------------------------------------------------

    def bdh(
        self,
        strSecurity: list[str],
        strFields: list[str],
        startdate: dt.datetime,
        enddate: dt.datetime,
        per: Literal[
            "DAILY", "MONTHLY", "QUARTERLY", "SEMIANNUALLY", "ANNUALLY"
        ] = "DAILY",
        perAdj: Literal["ACTUAL", "CALENDAR", "FISCAL"] = "CALENDAR",
        days: Literal[
            "NON_TRADING_WEEKDAYS", "ALL_CALENDAR_DAYS", "ACTIVE_DAYS_ONLY"
        ] = "ALL_CALENDAR_DAYS",
        fill: Literal["PREVIOUS_VALUE", "NIL_VALUE"] = "NIL_VALUE",
        curr: Literal["EUR", "USD", "JPY", "CHF", "GBP", "CAD", "AUD", "ZAR"] = "EUR",
    ) -> dict[str, pd.DataFrame]:
        """HistoricalDataRequest ;
            Gets historical data for a set of securities and fields

            Options can be selected these are outlined in “Reference Services and Schemas Guide.”

        Args:
            strSecurity (list[str]): list of str : list of tickers
            strFields (list[str]): list of fields, must be static fields (e.g. px_last instead of last_price)
            startdate (dt.datetime): date
            enddate (dt.datetime):
            per (str, optional): periodicitySelection; daily, monthly, quarterly, semiannually or annually. Defaults to "DAILY".
            perAdj (str, optional): periodicityAdjustment: ACTUAL, CALENDAR, FISCAL. Defaults to "CALENDAR".
            days (str, optional): nonTradingDayFillOption : NON_TRADING_WEEKDAYS*, ALL_CALENDAR_DAYS or ACTIVE_DAYS_ONLY. Defaults to "NON_TRADING_WEEKDAYS".
            fill (str, optional): nonTradingDayFillMethod :  PREVIOUS_VALUE, NIL_VALUE. Defaults to "PREVIOUS_VALUE".
            curr (str, optional): string, else default currency is used. Defaults to None.

        Returns:
            (int | list[pd.DataFrame]) : A list containing as many dataframes as requested fields
        # Partial response : 6
        # Response : 5
        """
        # -----------------------------------------------------------------------
        # Create request
        # -----------------------------------------------------------------------

        # Create request
        request = self.refDataSvc.createRequest("HistoricalDataRequest")

        # Put field and securities in list is single value is passed
        if isinstance(strFields, str):
            strFields = [strFields]

        if isinstance(strSecurity, str):
            strSecurity = [strSecurity]


        # Append list of securities
        for strF in strFields
            request.append("fields", strF)

        for strS in strSecurity:
            request.append("securities", strS)

        # Set other parameters
        request.set("startDate", startdate.strftime("%Y%m%d"))
        request.set("endDate", enddate.strftime("%Y%m%d"))
        request.set("periodicitySelection", per)
        request.set("periodicityAdjustment", perAdj)
        # request.set("nonTradingDayFillMethod", days)
        # request.set("nonTradingDayFillOption", fill)
        request.set("currency", curr)

        # -----------------------------------------------------------------------
        # Send request
        # -----------------------------------------------------------------------
        requestID = self.session.sendRequest(request)
        print("Sending request")

        # -----------------------------------------------------------------------
        # Receive request
        # -----------------------------------------------------------------------

        dict_Security_Fields = {}
        while True:
            event = self.session.nextEvent()

            # Ignores anything that's not partial or final

            if (event.eventType() != blpapi.event.Event.RESPONSE) & (
                event.eventType() != blpapi.event.Event.PARTIAL_RESPONSE
            ):
                continue
            # Extract the response message
            for msg in blpapi.event.MessageIterator(event):
                ticker = (
                    msg.getElement("securityData").getElement("security").getValue()
                )
                rows = []
                columns = []
                for i, fieldData in enumerate(
                    msg.getElement("securityData").getElement("fieldData")
                ):
                    if i == 0:
                        columns = [field.name() for field in fieldData]
                    rows.append([field.getValue() for field in fieldData])

                dict_Security_Fields[ticker] = pd.DataFrame(rows, columns=columns)
                if "date" in dict_Security_Fields[ticker].columns:
                    dict_Security_Fields[ticker] = dict_Security_Fields[
                        ticker
                    ].set_index("date")

            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break
        # -----------------------------------------------------------------------
        # Exploit data
        # -----------------------------------------------------------------------

        return dict_Security_Fields

    # ----------------------------------------------------------------------------------------------------       
    def __del__(self):
        print("Session closed")
        self.session.stop()

# Data processing class

In [239]:
class BbgDAO:
    def __init__(self, bbg_result: dict[str, pd.DataFrame]) -> None:
        """Constructor of the Bloomberg Data Access Object class

        Args:
        ----
            bbg_result (dict[str, pd.DataFrame]): The result from bloomberg in form of a dict of dataframe
        """
        self.__merged_dataframe = pd.concat(
            [v for v in bbg_result.values()], keys=tuple(bbg_result.keys()), axis=1
        )

    @property
    def full_dataframe(self) -> pd.DataFrame:
        """Get the raw dataframe merged from the bloomberg result

        Returns:
        ----
            pd.DataFrame: The raw dataframe merged from the bloomberg result
        """
        return self.__merged_dataframe

    @property
    def tickers(self) -> set[str]:
        """Get the available tickers

        Returns:
        ----
            set[str]: The set of tickers
        """
        return set(map(lambda x: x[0], self.__merged_dataframe.columns))

    @property
    def fields(self) -> set[str]:
        """Get the available fields

        Returns:
        ----
            set[str]: The set of fields
        """
        return set(map(lambda x: str(x[-1]), self.__merged_dataframe.columns))

    def __check_ticker(self, ticker_to_check: str) -> None:
        """Private checking method for the tickers

        Args:
            ticker_to_check (str): Raise an error in case of missing ticker
        """
        assert ticker_to_check in self.tickers, "Error provide a valid ticker"

    def __check_field(self, field_to_check: str) -> None:
        """Private checking method for the fields

        Args:
            field_to_check (str): Raise an error in case of missing field
        """
        assert field_to_check in self.fields, "Error provide a valid field"

    def one_ticker_all_fields(self, ticker: str) -> pd.DataFrame:
        """Extract all field for a given ticker

        Args:
        ----
            ticker (str): The ticker to extract

        Returns:
        ----
            pd.DataFrame: The pandas dataframe containing the fields for one ticker
        """
        self.__check_ticker(ticker)
        return self.__merged_dataframe.iloc[
            :, self.__merged_dataframe.columns.get_level_values(0) == ticker
        ]

    def one_ticker_one_field(self, ticker: str, field: str) -> pd.DataFrame:
        """Get one field for one ticker

        Args:
        ----
            ticker (str): The ticker selected
            field (str): The field for that ticker

        Returns:
        ----
            pd.DataFrame: The pandas dataframe containing the field for the ticker
        """
        self.__check_ticker(ticker)
        self.__check_field(field)
        return self.__merged_dataframe.iloc[
            :,
            (self.__merged_dataframe.columns.get_level_values(1) == field)
            & (self.__merged_dataframe.columns.get_level_values(0) == ticker),
        ]

    def all_ticker_one_field(self, field: str) -> pd.DataFrame:
        """Get the same field for all tickers

        Args:
        ----
            field (str): The desired field

        Returns:
        ----
            pd.DataFrame: The dataframe with the tickers and the correspond field
        """
        self.__check_field(field)
        return self.__merged_dataframe.iloc[
            :, self.__merged_dataframe.columns.get_level_values(1) == field
        ]

# IV. Tests


In [246]:
blp = BLP()
strFields = ["PX_LAST", "PX_VOLUME"]
tickers = ["GLE FP Equity", "TTE FP Equity"]
startDate = dt.datetime(2020, 10, 1)
endDate = dt.datetime(2020, 11, 3)
prices = blp.bdh(
    strSecurity=tickers, strFields=strFields, startdate=startDate, enddate=endDate
)
prices

Session open
Sending request


{'GLE FP Equity':             PX_LAST
 date               
 2020-10-01   11.048
 2020-10-02   11.038
 2020-10-05   11.388
 2020-10-06   12.152
 2020-10-07   12.336
 2020-10-08   12.594
 2020-10-09   12.416
 2020-10-12   12.712
 2020-10-13   12.210
 2020-10-14   12.322
 2020-10-15   11.826
 2020-10-16   12.058
 2020-10-19   12.296
 2020-10-20   12.608
 2020-10-21   12.392
 2020-10-22   12.508
 2020-10-23   12.750
 2020-10-26   12.584
 2020-10-27   11.944
 2020-10-28   11.382
 2020-10-29   11.350
 2020-10-30   11.640
 2020-11-02   12.124
 2020-11-03   12.784,
 'TTE FP Equity':             PX_LAST
 date               
 2020-10-01   28.490
 2020-10-02   28.210
 2020-10-05   28.855
 2020-10-06   29.520
 2020-10-07   29.170
 2020-10-08   29.495
 2020-10-09   29.950
 2020-10-12   29.680
 2020-10-13   29.210
 2020-10-14   29.170
 2020-10-15   28.215
 2020-10-16   28.455
 2020-10-19   28.355
 2020-10-20   28.230
 2020-10-21   27.775
 2020-10-22   27.620
 2020-10-23   28.080
 2020-10-26   27.055

In [241]:
d = BbgDAO(prices)
d.tickers, d.fields

({'GLE FP Equity', 'TTE FP Equity'}, {'PX_LAST', 'PX_VOLUME'})

In [242]:
d.one_ticker_all_fields("GLE FP Equity")

GLE FP Equity            
                 PX_LAST   PX_VOLUME
date                                
2020-10-01        11.048   5056724.0
2020-10-02        11.038   4921253.0
2020-10-05        11.388   4262064.0
2020-10-06        12.152   9123734.0
2020-10-07        12.336   8280475.0
2020-10-08        12.594   4990595.0
2020-10-09        12.416   4506467.0
2020-10-12        12.712   6560510.0
2020-10-13        12.210   6225036.0
2020-10-14        12.322   4234361.0
2020-10-15        11.826   6236560.0
2020-10-16        12.058   6586723.0
2020-10-19        12.296   2227175.0
2020-10-20        12.608   7325655.0
2020-10-21        12.392   5132329.0
2020-10-22        12.508   3928345.0
2020-10-23        12.750   5270262.0
2020-10-26        12.584   6023197.0
2020-10-27        11.944   6990562.0
2020-10-28        11.382  10900173.0
2020-10-29        11.350   6352473.0
2020-10-30        11.640   7073000.0
2020-11-02        12.124   6029462.0
2020-11-03        12.784   7722893.0

In [243]:
d.all_ticker_one_field("PX_VOLUME")

,GLE FP Equity,TTE FP Equity
,PX_VOLUME,PX_VOLUME
date,,
2020-10-01,5056724.0,8283681.0
2020-10-02,4921253.0,7680559.0
2020-10-05,4262064.0,6273524.0
2020-10-06,9123734.0,6705358.0
2020-10-07,8280475.0,5329794.0
2020-10-08,4990595.0,6220680.0
2020-10-09,4506467.0,5783149.0
2020-10-12,6560510.0,5525995.0


In [244]:
d.one_ticker_one_field("GLE FP Equity", "PX_LAST")

,GLE FP Equity
,PX_LAST
date,
2020-10-01,11.048
2020-10-02,11.038
2020-10-05,11.388
2020-10-06,12.152
2020-10-07,12.336
2020-10-08,12.594
2020-10-09,12.416
2020-10-12,12.712
